In [3]:
import onnxruntime as rt
import uproot
import numpy as np
from matplotlib import pyplot as plt

columns = [
    "era",
    "higgs1_reco_pt",
    "higgs1_reco_eta",
    "higgs1_reco_phi",
    "higgs1_reco_mass",
    "higgs2_reco_pt",
    "higgs2_reco_eta",
    "higgs2_reco_phi",
    "higgs2_reco_mass",
    "HT",
    "higgs1_DeltaRjj",
    "higgs2_DeltaRjj",
    "minDeltaR_Higgjj",
    "maxDeltaR_Higgjj",
    "higgs1_helicityCosTheta",
    "higgs2_helicityCosTheta",
    "hh_CosThetaStar_CS",
    "hh_vec_mass",
    "hh_vec_pt",
    "hh_vec_eta",
    "hh_vec_DeltaR",
    "hh_vec_DeltaPhi",
    "hh_vec_DeltaEta",
    "higgs1_reco_jet1_pt",
    "higgs1_reco_jet1_eta",
    "higgs1_reco_jet1_phi",
    "higgs1_reco_jet1_mass",
    "higgs1_reco_jet2_pt",
    "higgs1_reco_jet2_eta",
    "higgs1_reco_jet2_phi",
    "higgs1_reco_jet2_mass",
    "higgs2_reco_jet1_pt",
    "higgs2_reco_jet1_eta",
    "higgs2_reco_jet1_phi",
    "higgs2_reco_jet1_mass",
    "higgs2_reco_jet2_pt",
    "higgs2_reco_jet2_eta",
    "higgs2_reco_jet2_phi",
    "higgs2_reco_jet2_mass",
    "add_jet1pt_pt",
    "add_jet1pt_eta",
    "add_jet1pt_phi",
    "add_jet1pt_mass",
    "sigma_over_higgs1_reco_mass",
    "sigma_over_higgs2_reco_mass",
]



In [9]:
input_root=f'/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/file_root/JetMET_2022EE_2b_signal_region_to_4b_soumya_january2025.root'
tree=uproot.open(input_root)["tree"]
print(tree.keys())
input_data_dict = tree.arrays(columns, library="np")
n_events = 100
# get the input data as a numpy array
input_data = np.array([input_data_dict[col][:n_events] for col in columns], dtype=np.float32).T


['era', 'event', 'HT', 'jet_pt', 'jet_pt_raw', 'jet_eta', 'jet_phi', 'jet_mass', 'jet_mass_raw', 'jet_deepjet_probBVsAll', 'jet_pnet_probBVsAll', 'jet_pnet_probTauVsJet', 'jet_pnet_probb', 'jet_pnet_probc', 'jet_pnet_probuds', 'jet_pnet_probtauh', 'jet_pnet_ptcorr', 'jet_pnet_ptnu', 'jet_pnet_ptres', 'jet_pnet_jec_L1L2L3', 'jet_pnet_jec_L2Rel', 'jet_pnet_jec_L2L3Res', 'btag_cat', 'weight', 'is_boosted', 'weight_dnn', 'sigma_higgs1', 'sigma_over_higgs1_reco_mass', 'higgs1_reco_pt', 'higgs1_reco_eta', 'higgs1_reco_phi', 'higgs1_reco_mass', 'sigma_higgs2', 'sigma_over_higgs2_reco_mass', 'higgs2_reco_pt', 'higgs2_reco_eta', 'higgs2_reco_phi', 'higgs2_reco_mass', 'hh_vec_mass', 'hh_vec_pt', 'hh_vec_eta', 'hh_vec_phi', 'hh_vec_DeltaR', 'hh_vec_DeltaPhi', 'hh_vec_DeltaEta', 'hh_vec_ptOmass', 'njet', 'higgs1_reco_jet1_pt', 'higgs1_reco_jet1_eta', 'higgs1_reco_jet1_phi', 'higgs1_reco_jet1_mass', 'higgs1_reco_jet2_pt', 'higgs1_reco_jet2_eta', 'higgs1_reco_jet2_phi', 'higgs1_reco_jet2_mass', 'hig

In [56]:
model_path1='/work/mmalucch/out_ML_pytorch/DNN_AN_1e-3_e20drop75_minDelta1em5_SPANet_oversample_split_20folds/batch06/best_models/average_model_from_onnx.onnx'
# model_path1='/work/mmalucch/out_ML_pytorch/DNN_AN_1e-3_e20drop75_minDelta1em5_run2/state_dict/average_model_from_onnx.onnx'
# model_path1='/work/mmalucch/out_ML_pytorch/DNN_AN_VR1_1e-3_e20drop75_minDelta1em5_SPANet_oversample_split/state_dict/average_model_from_onnx.onnx'
# model_path2='/t3home/mmalucch/AnalysisConfigs/configs/VBF_HH4b/params/DNN_AN_1e-3_e20drop75_minDelta1em5_SPANet_noEarlyStopping_average_model_from_onnx.onnx'
model_path2='/t3home/mmalucch/AnalysisConfigs/configs/HH4b_common/params_common/DNN_AN_1e-3_e20drop75_minDelta1em5_SPANet_noEarlyStopping_20folds_average_model_from_onnx.onnx'

In [57]:
session_combine1 = rt.InferenceSession(
    model_path1,
    providers=rt.get_available_providers()
)
# print the input/putput name and shape
input_name=[input.name for input in session_combine1.get_inputs()]
output_name=[output.name for output in session_combine1.get_outputs()]
print("Inputs name:", input_name)
print("Outputs name:", output_name)

input_shape=[input.shape for input in session_combine1.get_inputs()]
output_shape=[output.shape for output in session_combine1.get_outputs()]
print("Inputs shape:", input_shape)
print("Outputs shape:", output_shape)

input_example = {input_name[0]: input_data}

output1 = session_combine1.run(output_name, input_example)[0]


Inputs name: ['args_0']
Outputs name: ['avg_w']
Inputs shape: [['N', 45]]
Outputs shape: [[None]]


In [58]:
session_combine2 = rt.InferenceSession(
    model_path2,
    providers=rt.get_available_providers()
)
# print the input/putput name and shape
input_name=[input.name for input in session_combine2.get_inputs()]
output_name=[output.name for output in session_combine2.get_outputs()]
print("Inputs name:", input_name)
print("Outputs name:", output_name)

input_shape=[input.shape for input in session_combine2.get_inputs()]
output_shape=[output.shape for output in session_combine2.get_outputs()]
print("Inputs shape:", input_shape)
print("Outputs shape:", output_shape)

input_example = {input_name[0]: input_data}

output2 = session_combine2.run(output_name, input_example)[0]


Inputs name: ['args_0']
Outputs name: ['avg_w']
Inputs shape: [['N', 45]]
Outputs shape: [[None]]


In [59]:
print(output1)
print(output2)


[0.27965802 0.43866467 1.1454651  1.925614   1.5226586  2.68134
 0.3624192  1.5703909  1.3618013  0.5576994  0.85620594 0.55731666
 1.9236794  0.8410357  0.9193077  2.2925465  1.7728382  0.60622454
 1.8308227  1.8210455  2.868959   1.3101743  5.466365   2.0104175
 3.9344602  3.7799087  0.7849596  0.7908114  2.9562385  0.42568263
 1.4923697  2.8888268  0.41120863 1.5220451  2.2313254  0.53853935
 1.457257   2.5520692  0.45866108 1.7270066  0.06353624 0.252005
 1.5893307  1.6388766  2.407554   0.08274977 3.8278847  0.25725943
 0.9874468  0.934441   1.416949   1.3407155  2.950825   4.7808046
 0.31878003 1.8980751  0.02493888 0.13207792 3.9670384  1.7567456
 0.0300223  2.3227136  2.590752   0.724976   1.1928934  0.5714618
 1.54893    0.4915843  1.9494274  3.1494994  3.3299866  1.8699617
 0.45878714 1.924846   2.7935812  3.0573123  2.460542   4.36362
 3.8764434  0.38924167 3.5186584  2.4178038  1.6948764  2.0390036
 6.1877675  3.2216973  2.186431   4.5110836  0.89834386 0.25345027
 3.576321